In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
pip install evaluate jiwer

In [ ]:
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import evaluate
import time
import pandas as pd

In [ ]:
common_voice = load_dataset("mozilla-foundation/common_voice_14_0", "ru", split="test", trust_remote_code=True)
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
samples = common_voice.select(range(100))

In [ ]:
models_info = {
    "openai/whisper-tiny": 39,
    "openai/whisper-base": 74,
    "openai/whisper-small": 244,
    "openai/whisper-medium": 769,
}

In [ ]:
wer_metric = evaluate.load("wer")
device = "cuda" if torch.cuda.is_available() else "cpu"
results = []

In [ ]:
for model_name, num_params_million in models_info.items():
    print(f"\n Evaluating {model_name}...")
        
    model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)
    model.eval()

    predictions = []
    references = []

    start_time = time.time()

    for example in samples:
        inputs = processor(
            example["audio"]["array"],
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features.to(device)

        with torch.no_grad():
            generated_ids = model.generate(inputs)

        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].lower()
        reference = example["sentence"].lower()

        predictions.append(transcription)
        references.append(reference)

    end_time = time.time()
    wer = wer_metric.compute(predictions=predictions, references=references)
    elapsed = end_time - start_time
    avg_time = elapsed / len(samples)

    results.append({
        "Model": model_name.split("/")[-1],
        "Params (M)": num_params_million,
        "WER": round(wer, 4),
        "Total Time (s)": round(elapsed, 2),
        "Time per Sample (s)": round(avg_time, 3)
    })

In [ ]:
df = pd.DataFrame(results)
df_sorted = df.sort_values("WER")

In [ ]:
print(df_sorted)

In [ ]:
df_sorted.plot(
    kind="barh",
    x="Model",
    y=["WER"],
    title="Whisper Model Comparison (WER & Speed)",
    figsize=(10,5),
    alpha=0.8
)

In [ ]:
df.to_csv('/kaggle/working/test.csv', index=False)